<a href="https://colab.research.google.com/github/Matthew-Fu/python-random-quote/blob/master/PS1_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

A helpful resource for getting up to speed with vector representations of documents is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [ ]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np

In [ ]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [ ]:
# NB: The current contents are for testing only
# This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  classes = []
  raw_text = []
  for cor in corpus:
    classes.append(cor["class"])
    raw_text.append(cor["text"])
  one_hot_vectorizer = CountVectorizer(binary=True)
  texts = one_hot_vectorizer.fit_transform(raw_text)
  #text = u" ".join(raw_text)
  #tokenizer = TweetTokenizer()
  #vocab = tokenizer.tokenize(text.lower())
  vocab = one_hot_vectorizer.get_feature_names()
  return texts,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here
  classes = []
  raw_text = []
  for cor in corpus:
    classes.append(cor["class"])
    raw_text.append(cor["text"])
  one_hot_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
  texts = one_hot_vectorizer.fit_transform(raw_text)
  #text = u" ".join(raw_text)
  #tokenizer = TweetTokenizer()
  #vocab = tokenizer.tokenize(text.lower())
  vocab = one_hot_vectorizer.get_feature_names()
  return texts,classes,vocab

In [ ]:
# given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [ ]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [ ]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.816500
The most informative terms for pos are: ['memorable', 'wonderfully', 'flaws', 'terrific', 'others', 'visually', 'hilarious', 'flawed', 'class', 'surprised', 'equally', 'color', 'pace', 'excellent', 'solid', 'today', 'perfectly', 'allows', 'overall', 'breathtaking']
The most informative terms for neg are: ['awful', 'mess', 'worst', 'waste', 'lame', 'laughable', 'boring', 'tedious', 'headed', 'cheap', 'titular', 'ridiculous', 'adam', 'supposed', 'potential', 'embarrassing', 'poor', 'nothing', 'terrible', 'alternate']
----------L2 Norm-----------
The model's average accuracy is 0.855500
The most informative terms for pos are: ['memorable', 'others', 'fun', 'hilarious', 'terrific', 'excellent', 'performances', 'true', 'pace', 'overall', 'very', 'wonderfully', 'sometimes', 'great', 'flaws', 'especially', 'well', 'american', 'perfectly', 'best']
The most informative terms for neg are: ['bad', 'worst', 'nothing', 'awful', '

Run the following to train and evaluate two models using extended features:

In [ ]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.820500
The most informative terms for pos are: ['memorable', 'terrific', 'wonderfully', 'flaws', 'hilarious', 'pace', 'visually', 'help of', 'equally', 'even if', 'others', 'overall', 'perfectly', 'thanks to', 'edge', 'history', 'deserves', 'films the', 'allows', 'sometimes']
The most informative terms for neg are: ['awful', 'mess', 'worst', 'boring', 'ridiculous', 'lame', 'headed', 'flat', 'unfortunately', 'studio', 'laughable', 'poor', 'to work', 'embarrassing', 'terrible', 'potential', 'ends up', 'nothing', 'bland', 'have']
----------L2 Norm-----------
The model's average accuracy is 0.872000
The most informative terms for pos are: ['very', 'great', 'he is', 'hilarious', 'well', 'also', 'fun', 'others', 'but it', 'performances', 'memorable', 'many', 'both', 'one of', 'seen', 'yet', 'as the', 'terrific', 'and it', 'overall']
The most informative terms for neg are: ['bad', 'nothing', 'worst', 'plot', 'unfortunately', 'bori